In [ ]:

# Conversor BPA Colab - carmea2025/conversor-colab

import pandas as pd
import io
import requests
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# URLs dos arquivos no GitHub
url_mapeamento = "https://raw.githubusercontent.com/carmea2025/conversor-colab/main/Mapeamento_Colunas.xlsx"
url_modelo = "https://raw.githubusercontent.com/carmea2025/conversor-colab/main/PLANO%20DE%20A%C3%87%C3%83O%20-%20BPA%20-%20FINAL.xlsx"

def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_arquivo, "wb") as f:
            f.write(r.content)
        print(f"✅ {nome_arquivo} baixado com sucesso.")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo}. Verifique o link do GitHub.")

# Baixa os arquivos automaticamente do GitHub
baixar_arquivo(url_mapeamento, "Mapeamento_Colunas.xlsx")
baixar_arquivo(url_modelo, "PLANO_DE_ACAO_BPA_FINAL.xlsx")

def converter_arquivo(uploaded_file):
    try:
        mapeamento = pd.read_excel("Mapeamento_Colunas.xlsx")
        modelo = pd.read_excel("PLANO_DE_ACAO_BPA_FINAL.xlsx")

        nome_arquivo = list(uploaded_file.keys())[0]
        conteudo = uploaded_file[nome_arquivo]
        df_bpa = pd.read_excel(io.BytesIO(conteudo))

        col_map = dict(zip(mapeamento['Coluna BPA'], mapeamento['Coluna Final']))
        df_convertido = df_bpa.rename(columns=col_map)

        colunas_finais = [col for col in modelo.columns if col in df_convertido.columns]
        df_final = df_convertido[colunas_finais]

        nome_saida = "BPA_Convertido.xlsx"
        df_final.to_excel(nome_saida, index=False)
        print("✅ Conversão concluída com sucesso!")
        files.download(nome_saida)
    except Exception as e:
        print(f"❌ Erro durante a conversão: {e}")

def interface():
    upload_button = widgets.FileUpload(accept=".xlsx", multiple=False)
    convert_button = widgets.Button(description="Converter e Baixar", button_style="success")

    def ao_clicar(_):
        clear_output(wait=True)
        display(upload_button, convert_button)
        if upload_button.value:
            converter_arquivo(upload_button.value)
        else:
            print("⚠️ Envie um arquivo BPA primeiro.")

    convert_button.on_click(ao_clicar)
    display(upload_button, convert_button)

interface()
